In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import pickle

In [3]:
pd.set_option("display.precision", 4)

In [4]:
Returns = pd.read_csv('Returns.csv')

In [5]:
Returns

,Date,MMM,MSFT,JPM,PLD,AMZN,XOM,JNJ,ATT,1Month
0,2019-07-15,0.0079,0.0345,0.0236,0.0170,-0.0109,-0.0024,0.0032,0.0415,2.17
1,2019-07-22,-0.0197,-0.0314,-0.0283,-0.0195,-0.0617,-0.0409,0.0026,0.0006,2.13
2,2019-07-29,-0.0415,0.0059,-0.0282,0.0164,-0.0086,-0.0127,0.0074,0.0108,2.12
3,2019-08-05,-0.0121,-0.0115,-0.0184,0.0068,-0.0083,-0.0359,-0.0051,0.0124,2.07
4,2019-08-12,-0.0262,-0.0169,-0.0158,-0.0061,-0.0240,0.0004,-0.0276,-0.0043,2.09
...,...,...,...,...,...,...,...,...,...,...
256,2024-06-10,0.0148,0.0163,0.0130,-0.0257,0.0295,0.0151,0.0221,0.0431,5.47
257,2024-06-17,-0.0020,-0.0063,0.0304,0.0355,0.0221,0.0394,-0.0174,0.0386,5.45
258,2024-06-24,-0.0085,0.0461,0.0125,0.0191,0.0349,-0.0152,0.0022,-0.0178,5.42
259,2024-07-01,0.0268,-0.0300,0.0063,0.0497,-0.0275,-0.0009,0.0232,0.0021,5.48


In [6]:
n_runs = 20
n_stock = 5
seq_len = 4
n_iter = 2000
batch = 10

outRtn_dates = []

for scene in range(n_runs):
    scene = scene*10
    out_of_sample_idx = scene + batch + seq_len
    outRtn_dates.append(Returns.iloc[out_of_sample_idx]['Date'])

In [7]:
outRtn_dates

['2019-10-21',
 '2019-12-30',
 '2020-03-09',
 '2020-05-18',
 '2020-07-27',
 '2020-10-05',
 '2020-12-14',
 '2021-02-22',
 '2021-05-03',
 '2021-07-12',
 '2021-09-20',
 '2021-11-29',
 '2022-02-07',
 '2022-04-18',
 '2022-06-27',
 '2022-09-05',
 '2022-11-14',
 '2023-01-23',
 '2023-04-03',
 '2023-06-12']

In [8]:
out_rates = Returns[Returns['Date'].isin(outRtn_dates)]['1Month'].values

In [9]:
out_rates

array([1.76, 1.51, 0.57, 0.1 , 0.1 , 0.09, 0.07, 0.03, 0.02, 0.05, 0.06,
       0.07, 0.03, 0.38, 1.16, 2.45, 3.72, 4.69, 4.7 , 5.24])

## SCOT Method

In [10]:
SCOT_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_SCOT_radi_{}_risk_{}/DRORtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            SCOT_Rtn = pickle.load(fp)
        SCOT_Results[idx, :] = SCOT_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, SCOT_Rtn.mean()/SCOT_Rtn.std())
        print(SCOT_Rtn.mean(), SCOT_Rtn.std())

Radius 0.05 Risk aversion 0.0 18.333828071532213
1.0218985243822711 0.05573841537049321
Radius 0.1 Risk aversion 0.0 18.429191431378026
1.0225107773604774 0.055483214289017785
Radius 0.2 Risk aversion 0.0 18.174804727825126
1.0202998485743182 0.056138146398473664
Radius 0.05 Risk aversion 0.1 21.600879963901285
1.0362651158433418 0.04797328245771078
Radius 0.1 Risk aversion 0.1 21.107104745791755
1.0305976641712515 0.048827050255517764
Radius 0.2 Risk aversion 0.1 18.18643650036659
1.0115387938598484 0.05562050563558498
Radius 0.05 Risk aversion 1.0 20.729889906443155
1.0273922358771064 0.04956091134655654
Radius 0.1 Risk aversion 1.0 19.278927124954155
1.0188521673308333 0.05284797025930228
Radius 0.2 Risk aversion 1.0 17.846970458130766
1.0170872929013546 0.05698935263480461


In [11]:
SCOT_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        SCOT_df.loc[idx, 'RiskAversion'] = risk_aver
        SCOT_df.loc[idx, 'Radius'] = radi
        ExcessReturn = SCOT_Results[idx, :] - out_rates/12*0.01 - 1
        SCOT_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
        SCOT_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
        SCOT_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
        idx += 1

In [12]:
SCOT_df

,RiskAversion,Radius,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.05,0.0208,0.0561,1.2829
1,0.0,0.10,0.0214,0.0559,1.3269
2,0.0,0.20,0.0192,0.0566,1.1750
3,0.1,0.05,0.0351,0.0484,2.5165
4,0.1,0.10,0.0295,0.0492,2.0754
5,0.1,0.20,0.0104,0.0560,0.6449
6,1.0,0.05,0.0263,0.0499,1.8234
7,1.0,0.10,0.0177,0.0531,1.1564
8,1.0,0.20,0.0160,0.0573,0.9652


In [12]:
SCOT_df.to_csv('SCOT_df.csv', header=True, index=False)

## Non-robust method

In [13]:
Non_Results = np.zeros((3, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    logdir = './logs/mv_Nonrobust_risk_{}/DRORtn.pickle'.format(risk_aver)
    with open(logdir, 'rb') as fp:
        Non_Rtn = pickle.load(fp)
    Non_Results[idx, :] = Non_Rtn
    idx += 1
    print('Risk aversion', risk_aver, Non_Rtn.mean()/Non_Rtn.std())
    print(Non_Rtn.mean(), Non_Rtn.std())

Risk aversion 0.0 16.47121294713709
1.0209980991982672 0.061986819214533306
Risk aversion 0.1 16.89390594292983
1.0253407014950717 0.06069293299955781
Risk aversion 1.0 17.684888723819785
1.0235186634700792 0.05787532392508082


In [14]:
Non_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    Non_df.loc[idx, 'RiskAversion'] = risk_aver
    ExcessReturn = Non_Results[idx, :] - out_rates/12*0.01 - 1
    Non_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
    Non_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
    Non_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
    idx += 1

In [15]:
Non_df

,RiskAversion,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.0199,0.0624,1.1033
1,0.1,0.0242,0.0610,1.3752
2,1.0,0.0224,0.0582,1.3342


In [16]:
Non_df.to_csv('Non_df.csv', header=True, index=False)

## Naive method (As a sanity check, it should be the same while risk aversion/radius is different)

In [16]:
Naive_Results = np.zeros((3, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    logdir = './logs/mv_Nonrobust_risk_{}/NaiveRtn.pickle'.format(risk_aver)
    with open(logdir, 'rb') as fp:
        Naive_Rtn = pickle.load(fp)
    Naive_Results[idx, :] = Naive_Rtn
    idx += 1
    print('Risk aversion', risk_aver, Naive_Rtn.mean()/Naive_Rtn.std())
    print(Naive_Rtn.mean(), Naive_Rtn.std())

Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972


In [17]:
Naive_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_SCOT_radi_{}_risk_{}/NaiveRtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            Naive_Rtn = pickle.load(fp)
        Naive_Results[idx, :] = Naive_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, Naive_Rtn.mean()/Naive_Rtn.std())
        print(Naive_Rtn.mean(), Naive_Rtn.std())

Radius 0.05 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 0.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.05 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 0.1 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.05 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.1 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972
Radius 0.2 Risk aversion 1.0 19.568668673641902
1.0110756796712426 0.05166808721295972


In [18]:
Naive_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    Naive_df.loc[idx, 'RiskAversion'] = risk_aver
    ExcessReturn = Naive_Results[idx, :] - out_rates/12*0.01 - 1
    Naive_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
    Naive_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
    Naive_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
    idx += 1

In [19]:
Naive_df

,RiskAversion,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.01,0.052,0.6633
1,0.1,0.01,0.052,0.6633
2,1.0,0.01,0.052,0.6633


In [21]:
Naive_df.to_csv('Naive_df.csv', header=True, index=False)

## OT Method

In [20]:
OT_Results = np.zeros((9, 20))
idx = 0
for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        logdir = './logs/mv_OT_radi_{}_risk_{}/DRORtn.pickle'.format(radi, risk_aver)
        with open(logdir, 'rb') as fp:
            OT_Rtn = pickle.load(fp)
        OT_Results[idx, :] = OT_Rtn
        idx += 1
        print('Radius', radi, 'Risk aversion', risk_aver, OT_Rtn.mean()/OT_Rtn.std())
        print(OT_Rtn.mean(), OT_Rtn.std())

Radius 0.05 Risk aversion 0.0 19.444214813468562
1.011318461272276 0.052011277954600615
Radius 0.1 Risk aversion 0.0 18.87868219814078
1.0127894715060966 0.05364725465879385
Radius 0.2 Risk aversion 0.0 19.654746371978852
1.0188076094439746 0.051835194927595515
Radius 0.05 Risk aversion 0.1 19.928738899124102
1.0132159139999874 0.05084194836054176
Radius 0.1 Risk aversion 0.1 19.1814569160587
1.0130609592839426 0.052814599209917615
Radius 0.2 Risk aversion 0.1 20.834881023566513
1.0216407256868827 0.049035112057097714
Radius 0.05 Risk aversion 1.0 19.650365471254855
1.01105694010774 0.051452322430681735
Radius 0.1 Risk aversion 1.0 19.449945527717826
1.011221589782684 0.05199097284573919
Radius 0.2 Risk aversion 1.0 19.55179067472692
1.0127259463491458 0.05179709435301074


In [21]:
OT_df = pd.DataFrame()
idx = 0

for risk_aver in [0.0, 0.1, 1.0]:
    for radi in [0.05, 0.1, 0.2]:
        OT_df.loc[idx, 'RiskAversion'] = risk_aver
        OT_df.loc[idx, 'Radius'] = radi
        ExcessReturn = OT_Results[idx, :] - out_rates/12*0.01 - 1
        OT_df.loc[idx, 'ExcessReturn_Mean'] = ExcessReturn.mean()
        OT_df.loc[idx, 'ExcessReturn_STD'] = ExcessReturn.std()
        OT_df.loc[idx, 'ExcessReturn_Sharpe'] = ExcessReturn.mean()/ExcessReturn.std()*np.sqrt(12)
        
        idx += 1

In [22]:
OT_df

,RiskAversion,Radius,ExcessReturn_Mean,ExcessReturn_STD,ExcessReturn_Sharpe
0,0.0,0.05,0.0102,0.0523,0.6751
1,0.0,0.10,0.0117,0.0540,0.7486
2,0.0,0.20,0.0177,0.0523,1.1725
3,0.1,0.05,0.0121,0.0512,0.8189
4,0.1,0.10,0.0119,0.0532,0.7778
5,0.1,0.20,0.0205,0.0495,1.4362
6,1.0,0.05,0.0099,0.0518,0.6650
7,1.0,0.10,0.0101,0.0523,0.6688
8,1.0,0.20,0.0116,0.0522,0.7708


In [25]:
OT_df.to_csv('OT_df.csv', header=True, index=False)

In [ ]:
import torch

In [ ]:
x=torch.ones(2, requires_grad=True)
y=1.2*x
z=3+x.detach()
r=(y+z).sum()

r.backward()
x.grad

In [ ]:
z.grad